In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
#https://github.com/fraunhoferhhi/BerlinV2X
sidelink='/home/bde/Data/BerlinV2X/sidelink_dataframe.parquet'
cellular='/home/bde/Data/BerlinV2X/cellular_dataframe.parquet'
df = pd.read_parquet(cellular)
df.info(verbose=True)
df.head()

In [ ]:
df['measured_qos'].unique()
df_datarate = df[df['measured_qos'] == 'datarate']
#df_datarate.info(verbose=True)
df_delay = df[df['measured_qos'] == 'delay']
#df_delay.info(verbose=True)
df_delay.head()

In [ ]:
df_downlink = df[df['direction'] == 'downlink']
#df_downlink.info(verbose=True)
df_uplink = df[df['direction'] == 'uplink']
#df_uplink.info(verbose=True)

In [ ]:
df_pc1 = df[df['device'] == 'pc1']
df_pc2 = df[df['device'] == 'pc2']
df_pc3 = df[df['device'] == 'pc3']
df_pc4 = df[df['device'] == 'pc4']
#df_pc?.info()
selected_df = df_pc2

In [ ]:
#df = df.drop(columns=['device', 'ping_ms'])
df_new = selected_df[['datarate', 'PCell_RSRP_1','PCell_RSRP_2','PCell_RSRQ_1','PCell_RSRQ_2','PCell_RSSI_1','PCell_RSSI_2','PCell_SNR_1','PCell_SNR_2', 'operator']]
#df_new = selected_df[['datarate', 'PCell_RSRP_1','PCell_RSRP_2','PCell_RSRQ_1','PCell_RSRQ_2','PCell_RSSI_1','PCell_RSSI_2','PCell_SNR_1','PCell_SNR_2', 'operator',
#                      'SCell_RSRP_1','SCell_RSRP_2','SCell_RSRQ_1','SCell_RSRQ_2','SCell_RSSI_1','SCell_RSSI_2','SCell_SNR_1']]

#df_new = selected_df[['datarate', 'SCell_RSRP_1','SCell_RSRP_2','SCell_RSRQ_1','SCell_RSRQ_2','SCell_RSSI_1','SCell_RSSI_2','SCell_SNR_1', 'operator']]

df_new = df_new.dropna()
df_new.info()

In [ ]:
#df_new.head(100)
df_new.to_csv(f'data/BERLINV2X/test.csv', encoding='utf-8', index=False)
#df_new.describe()

In [ ]:
df_new['datetime'] = df_new.index

groups=(df_new['datetime'].diff()>pd.Timedelta(seconds=1)).cumsum()+1
max_len = 0
continous_groups = []
for i , group in df_new.groupby(groups):
    if len(group) > 2048:
        max_len = len(group)
        continous_groups.append(group)
        print(len(group))
for group in continous_groups:
    group.info()

In [ ]:
longest = continous_groups[0]
# 0, 3, 4, 5, 9, 10, 11, 12, 13
""" longest = pd.concat([continous_groups[0],
                    continous_groups[3],
                    continous_groups[4],
                    continous_groups[5],
                    continous_groups[9],
                    continous_groups[10],
                    continous_groups[11],
                    continous_groups[12],
                    continous_groups[13]]) """
longest.info()

In [ ]:
longest['date'] = longest['datetime']
longest = longest.drop(columns=['operator', 'datetime'])
longest.to_csv(f'data/BERLINV2X/longest.csv', encoding='utf-8', index=False)
longest.head()

In [ ]:
longest.info()

In [ ]:
# 0, 3, 4, 5, 9, 10, 11, 12, 13
SCOPE = 13
continous_groups[SCOPE][['datarate']].plot(figsize=(9,6))

In [ ]:
#df.query('"2014-07-23 07:55:00" <= datetime <= "2014-07-23 08:20:00"')
continous_groups[SCOPE][['PCell_RSRP_1', 'PCell_RSRQ_1', 'PCell_RSSI_1', 'PCell_SNR_1']].plot(figsize=(9,6))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,3, figsize=(20, 6))

sns.heatmap(longest.corr('pearson'), ax=ax[0], annot=True)
sns.heatmap(longest.corr('spearman'), ax=ax[1], annot=True)
sns.heatmap(longest.corr('kendall'), ax=ax[2], annot=True)

In [ ]:
def multi_corr(dataframe, feature):
    fig, ax = plt.subplots(1,3, figsize=(20, 6))

    # when the Pearson and Spearman values are not much different, 
    # our data tends to not have extreme values (outliers)
    corr1 = dataframe.corr('pearson')[[feature]].sort_values(by=feature, ascending=False)
    corr2 = dataframe.corr('spearman')[[feature]].sort_values(by=feature, ascending=False)
    #ordinal correlation (Spearman & Kendall Tau)
    corr3 = dataframe.corr('kendall')[[feature]].sort_values(by=feature, ascending=False)
    
    s1 = sns.heatmap(corr1, ax=ax[0], annot=True)
    #s1.set(xlabel='X-Axis', ylabel='Y-Axis')
    s1.set(xlabel='Pearson')
    s2 = sns.heatmap(corr2, ax=ax[1], annot=True)
    s2.set(xlabel='Spearman')
    s3 = sns.heatmap(corr3, ax=ax[2], annot=True)
    s3.set(xlabel='Kendall')

In [ ]:
# 0, 3, 4, 5, 9, 10, 11, 12, 13
multi_corr(continous_groups[0], 'datarate')